In [1]:
import itertools
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier

In [4]:
!pip install kaggle
from google.colab import drive
drive.mount('/content/drive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /content/drive/MyDrive/kaggle. Or use the environment method.


In [6]:
!kaggle datasets download -d houssemaminetouihri/godatascience2023

100% 305M/305M [00:16<00:00, 16.5MB/s]
100% 305M/305M [00:16<00:00, 19.2MB/s]


In [7]:
import zipfile

# Define the path to your zip file
file_path = '/content/drive/MyDrive/kaggle/godatascience2023.zip'  # Replace 'your_file.zip' with your file's name

# Unzip the file to a specific destination
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/kaggle')

In [19]:
train=pd.read_csv('/content/drive/MyDrive/kaggle/Train_one_hot/Train.csv')
test=pd.read_csv('/content/drive/MyDrive/kaggle/Test/Test.csv')
ss=pd.read_csv('/content/drive/MyDrive/kaggle/SampleSubmission.csv')

In [9]:
train.shape

(324099, 59)

In [20]:
train.isna().sum()

ID                                                    0
cert.alt                                         235327
cert.curve                                       314335
cert.hash                                        235289
cert.issuerCN                                    235293
cert.issuerON                                    235293
cert.notAfter                                    235289
cert.notBefore                                   235289
cert.publicAlgorithm                             235289
cert.remainingDays                               235289
cert.serial                                      235289
cert.subjectCN                                   235289
cert.subjectON                                   235403
cert.validDays                                   235289
ip.dsfield.dscp                                       0
ip.dsfield.ecn                                        0
ip.flags.df                                           0
ip.id                                           

In [22]:
#dropping columns with many missing values
l=[]
for x in train.columns:
  if train[x].isna().sum() >0:
    train.drop([x],axis=1,inplace=True)
for x in test.columns:
  if test[x].isna().sum() >0:
    test.drop([x],axis=1,inplace=True)


In [37]:
train.isna().sum()

y = train.iloc[:, train.columns.get_loc('linux_centos-7'):]

X = train.drop(y.columns, axis=1)
X.columns


Index(['ID', 'ip.dsfield.dscp', 'ip.dsfield.ecn', 'ip.flags.df', 'ip.id',
       'ip.len', 'ip.ttl', 'tcp.flags', 'tcp.offset',
       'tcp.options.timestamp.tsval', 'tcp.window_size_value', 'tls.cipher',
       'tls.handshake.ciphersuite.ch', 'tls.handshake.extension.len',
       'tls.handshake.extension.type', 'tls.handshake.extensions_alpn_str.ch',
       'tls.handshake.extensions_ec_point_format.ch',
       'tls.handshake.extensions_ec_point_format.sh',
       'tls.handshake.extensions_length',
       'tls.handshake.extensions_supported_group.ch',
       'tls.handshake.sig_hash_alg.ch', 'tls.handshake.type',
       'tls.handshake.version.ch', 'tls.handshake.version.sh', 'tls.ja3',
       'tls.ja3s', 'tls.record.version.ch', 'frame.time_epoch',
       'packet_directions'],
      dtype='object')

In [38]:
concat_df = pd.concat([X, test], axis=0)

In [39]:
#replacing each list by its length
import ast
from sklearn.preprocessing import LabelEncoder
def modify_value(value):
    if isinstance(value,str):
      x=len(value)
    else:
      x=value
    return x
all=pd.DataFrame()
for i in X.columns:
  X_new = concat_df[i].apply(modify_value)
  all=pd.concat([all.reset_index(drop=True),X_new.reset_index(drop=True)],axis=1)

In [40]:
#label encoding
all1=pd.DataFrame()
for i in X.columns:
    lbl=LabelEncoder()
    e=lbl.fit_transform(concat_df[i])
    e=pd.DataFrame(e,columns=[i+'_lbl'])
    all1=pd.concat([all1.reset_index(drop=True),e.reset_index(drop=True)],axis=1)

In [41]:
#concatinatin two dataframes, it will be used for the train and test
jemla=pd.concat([all,all1],axis=1)

In [42]:
tr = jemla.iloc[:X.shape[0], :]
te = jemla.iloc[X.shape[0]:, :]

In [43]:
te.head()

,ID,ip.dsfield.dscp,ip.dsfield.ecn,ip.flags.df,ip.id,ip.len,ip.ttl,tcp.flags,tcp.offset,tcp.options.timestamp.tsval,...,tls.handshake.extensions_supported_group.ch_lbl,tls.handshake.sig_hash_alg.ch_lbl,tls.handshake.type_lbl,tls.handshake.version.ch_lbl,tls.handshake.version.sh_lbl,tls.ja3_lbl,tls.ja3s_lbl,tls.record.version.ch_lbl,frame.time_epoch_lbl,packet_directions_lbl
324099,9,145,145,145,290,185,174,359,174,406,...,32,6,5534,1,2,64,41721,0,281430,212434
324100,9,80,80,80,160,105,96,186,96,174,...,20,6,3985,1,2,64,117659,0,472723,284328
324101,9,190,190,190,380,240,228,489,228,512,...,1,1,29999,1,1,29,11552,0,7820,133006
324102,9,130,130,130,260,171,156,353,156,352,...,14,6,5629,1,2,53,96546,0,407719,193059
324103,9,110,110,110,220,147,141,310,132,88,...,1,1,21485,1,1,29,11552,0,98748,6374


In [44]:
from sklearn.decomposition import PCA
pca=PCA(28)
X_pca=pca.fit_transform(tr)
Te_pca=pca.transform(te)

In [45]:
pca_train = pd.DataFrame(X_pca)
pca_test = pd.DataFrame(Te_pca)

In [46]:
df_train = pd.concat([tr, pca_train],axis=1)


In [47]:
df_test1 = pd.concat([te, pca_test],axis=1)

In [48]:
df_train.shape

(324099, 86)

In [49]:
pca_test.shape

(159631, 28)

In [50]:
len(id)

TypeError: object of type 'builtin_function_or_method' has no len()

In [51]:
# using randomforestclassifier on data generated by PCA

from sklearn.metrics import hamming_loss
from sklearn.model_selection import train_test_split
xt = RandomForestClassifier(n_estimators=100, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2)

xt.fit(X_train, y_train)

# predict on the test data
y_pred = xt.predict(X_test)
y_final=xt.predict(Te_pca)
print('os_hamming:',hamming_loss(y_test,y_pred))


KeyboardInterrupt: 

In [ ]:
#trying to predict each target individually and ensemble all models in a list model

import xgboost as xgb
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(tr, y, test_size=0.1, random_state=42)

# Train the XGBoost model
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 15,
    'learning_rate': 0.1,
    'n_estimators': 500,
    'n_jobs': -1
}

num_targets = y.shape[1]
model = []
for i in range(num_targets):
    y_train_i = y_train.iloc[:, i]
    y_test_i = y_test.iloc[:, i]
    dtrain = xgb.DMatrix(X_train, label=y_train_i)
    dtest = xgb.DMatrix(X_test, label=y_test_i)
    model_i = xgb.train(params, dtrain, num_boost_round=500,early_stopping_rounds=10, evals=[(dtest, 'test')])
    model.append(model_i)

In [ ]:
y_pred = np.zeros((159631,15))
for i in range(num_targets):
    y_pred_i = model[i].predict(xgb.DMatrix(te))
    y_pred[:, i] = np.round(y_pred_i)

y_pred

In [ ]:
#make submission
ss.iloc[:,1:] = y_pred

In [ ]:
ss

In [ ]:
ss.to_csv('AAA.csv',index=False)